In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
base_dir='/content/drive/My Drive/Colab/Hackathons/Emergency_vehicle/Data'
train_dir=os.path.join(base_dir,'train')
test_dir=os.path.join(base_dir,'test_data')
validation_dir=os.path.join(base_dir,'validation')

In [4]:
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.optimizers import RMSprop,adam,SGD
from keras.metrics import accuracy
#from keras.losses import binary_crossentropy
from keras.applications import VGG16,InceptionV3,Xception,ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras import regularizers

Using TensorFlow backend.


In [5]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(256, 256, 3))

conv_base.summary()

58892288/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

In [6]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 8, 8, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
    directory,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
train_features, train_labels = extract_features(train_dir, 1381)
validation_features, validation_labels = extract_features(validation_dir, 265)

Found 1381 images belonging to 2 classes.
Found 265 images belonging to 2 classes.


In [7]:
train_features = np.reshape(train_features, (1381, 8 * 8 * 512))
validation_features = np.reshape(validation_features, (265, 8 * 8 * 512))

from keras import models
from keras import layers
from keras import optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=8 * 8 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Train on 1381 samples, validate on 265 samples
Epoch 1/30
1381/1381 [==============================] - 1s 883us/step - loss: 0.5102 - acc: 0.7509 - val_loss: 0.4362 - val_acc: 0.7774
Epoch 2/30
1381/1381 [==============================] - 1s 674us/step - loss: 0.3365 - acc: 0.8624 - val_loss: 0.3026 - val_acc: 0.8679
Epoch 3/30
1381/1381 [==============================] - 1s 666us/step - loss: 0.2676 - acc: 0.8928 - val_loss: 0.4087 - val_acc: 0.8264
Epoch 4/30
1381/1381 [==============================] - 1s 662us/step - loss: 0.2208 - acc: 0.9131 - val_loss: 0.2822 - val_acc: 0.8830
Epoch 5/30
1381/1381 [==============================] - 1s 662us/step - loss: 0.1728 - acc: 0.9464 - val_loss: 0.2426 - val_acc: 0.8830
Epoch 6/30
1381/1381 [==============================] - 1s 663us/step - loss: 0.1416 - acc: 0.9508 - val_loss: 0.2882 - val_acc: 0.8868
Epoch 7/30
1381/1381 [==============================] - 1s 663us/step - loss: 0.1296 - acc: 0.9558 - val_loss: 0.2557 - val_acc: 0.9057
E

In [ ]:
#Data Preprocessing
train_datagen=ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=60,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.5,
                                 horizontal_flip=True
                                 )
val_datagen=ImageDataGenerator(rescale=1.0/255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(256,256),
                                                  batch_size=1381,
                                                  class_mode='binary',
                                                  shuffle=True,
                                                  color_mode="rgb")
validation_generator=val_datagen.flow_from_directory(validation_dir,
                                                     target_size=(256,256),
                                                     batch_size=265,
                                                     class_mode='binary',
                                                     shuffle=True,
                                                     color_mode="rgb")

Found 1381 images belonging to 2 classes.
Found 265 images belonging to 2 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
 print('data batch shape:', data_batch.shape)
 print('labels batch shape:', labels_batch.shape)
 print(data_batch[0])
 print(labels_batch[0])
 break

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)
#model.fit_generator(train_generator,steps_per_epoch=STEP_SIZE_TRAIN,epochs=50,validation_data=validation_generator,validation_steps=STEP_SIZE_VALID)

1
1


In [ ]:
model.fit_generator(train_generator,steps_per_epoch=STEP_SIZE_TRAIN,epochs=100,validation_data=validation_generator,validation_steps=STEP_SIZE_VALID)

Epoch 1/100


ResourceExhaustedError: ignored

In [ ]:
model.evaluate_generator(generator=validation_generator,steps=STEP_SIZE_VALID)

[0.4297305941581726, 0.82421875]

In [13]:
test_generator=datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    shuffle=False,
    class_mode=None,
    batch_size=1
)

test_features=conv_base.predict(test_generator)
test_features = np.reshape(test_features, (706, 8 * 8 * 512))
y_pred=model.predict(test_features)

Found 706 images belonging to 1 classes.


In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

706/706 [==============================] - 317s 448ms/step


In [14]:
y_pred.shape

(706, 1)

In [ ]:
y_pred

In [15]:
pred=[]
for i in y_pred:
  if(i<=0.5):
     pred.append(0)
  else:
    pred.append(1)

In [ ]:
pred

In [17]:
test_path='/content/drive/My Drive/Colab/Hackathons/Emergency_vehicle/Data/test.csv'
test_df=pd.read_csv(test_path)

In [18]:
test_df.head()

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [19]:
ind=[]
sort_ind=[]
for i in test_df.index:
  str1=test_df['image_names'][i].split('.')[0]
  ind.append(int(str1))
  sort_ind.append(int(str1))

In [ ]:
len(ind)

706

In [20]:
sort_ind.sort()

In [21]:
res = {sort_ind[i]: pred[i] for i in range(len(sort_ind))} 

In [22]:
sub_df = pd.DataFrame(columns=['image_names', 'emergency_or_not'])

for i in range(706):
  sub_df.loc[i,'image_names']=test_df['image_names'][i]
  sub_df.loc[i,'emergency_or_not']=res[ind[i]]

In [23]:
sub_df.head()

,image_names,emergency_or_not
0,1960.jpg,1
1,668.jpg,1
2,2082.jpg,0
3,808.jpg,1
4,1907.jpg,0


In [24]:
sub_df.to_csv('predictions.csv',index=False)